In [60]:
import time
from xml.dom import minidom

import json, pprint

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


In [61]:
base_url = 'https://discovery.splunkcloud.com:8089'
username = 'tableau_user'
password = '9d23f85964476d364672742c2dfeafd1'

output= 'csv'

In [62]:
search_query = '''
search index="streaming" api.request.uri="/v1/streaming/video/*" earliest=-1h@h latest=-1h@h+59m+59s
| spath path=api.response.body.vendorAttributes.upLynk.preplay.sid output=sid
| search sid=*
| spath path=api.request.headers.x_request_query_params output=api_query_params
| spath path=api.response.epoch output=epoch
| eval date=strftime(epoch,"%m/%d/%y")
| eval hour=strftime(epoch,"%m/%d/%y %H:00:00")
| eval api_query_params_temp = split(api_query_params, "&")
| eval platform_group = mvindex(split(mvindex(split(mvindex(api_query_params_temp, mvfind(api_query_params_temp, "adCsid=")), "="), 1),"%2F"),1)
| eval device_type = mvindex(split(mvindex(split(mvindex(api_query_params_temp, mvfind(api_query_params_temp, "adProfile=")), "="), 1),"_"),2)
| eval device_temp = mvindex(split(mvindex(split(mvindex(api_query_params_temp, mvfind(api_query_params_temp, "adCsid=")), "="), 1),"%2F"),2)
| eval device = case(device_type="chromecast", "chromcast", platform_group="web" and device_temp in ("","macos","desktop"), "desktop-web", platform_group="web" and device_temp in ("ios","android"), "mobile-web", 1=1,device_temp)
| eval adcaid = mvindex(split(mvindex(api_query_params_temp, mvfind(api_query_params_temp, "adCaid=")), "="), 1)
| eval brand = mvindex(split(mvindex(api_query_params_temp, mvfind(api_query_params_temp, "product=")), "="), 1)
| spath path=api.response.body.cuePoints{} output=midbreaks_avail
| eval midbreaks_avail=if(isnull(midbreaks_avail) or midbreaks_avail="null","",midbreaks_avail)
| eval midbreaks_avail_count=if(midbreaks_avail="",0,mvcount(midbreaks_avail))
| search midbreaks_avail_count>0
| spath path=uplynk.response.body.ads.breaks.ad_count output=ad_break_count
| eval zero_breaks = case(ad_break_count = 0, 1, 1=1, 0)
| eval some_breaks = case(ad_break_count > 0, 1, 1=1, 0)
| table date hour brand device zero_breaks some_breaks
| stats sum(zero_breaks) as zero_breaks sum(some_breaks) as some_breaks by date hour brand device
'''

In [63]:
# r = requests.get(base_url+"/servicesNS/admin/search/auth/login",
#                  data={'username':username,'password':password}, verify=False)

# session_key = minidom.parseString(r.text).getElementsByTagName('sessionKey')[0].firstChild.nodeValue
# print ("Session Key:", session_key)

In [88]:
# standard_library.install_aliases()
import urllib.request, urllib.parse, urllib.error
r = requests.post(base_url + '/services/search/jobs/', data=urllib.parse.urlencode({'search': search_query}),
                  auth=(username, password),
                  verify = False)
r.text

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n  <sid>1606766227.4333509_F5F42097-DD8F-4BA5-8BB2-DF2A822D40F2</sid>\n</response>\n'

In [89]:
sid = minidom.parseString(r.text).getElementsByTagName('sid')[0].firstChild.nodeValue
print ("Search ID", sid)

Search ID 1606766227.4333509_F5F42097-DD8F-4BA5-8BB2-DF2A822D40F2


In [90]:
# Search ID 1606759181.4360243_3967FE72-450A-4119-9892-B86439CFC34B

In [91]:
done = False
while not done:
    r = requests.get(base_url + '/services/search/jobs/' + sid,
        headers = { 'Authorization': ('Splunk %s' %session_key)},
        verify = False)
    response = minidom.parseString(r.text)
    for node in response.getElementsByTagName("s:key"):
        if node.hasAttribute("name") and node.getAttribute("name") == "dispatchState":
            dispatchState = node.firstChild.nodeValue
            print ("Search Status: ", dispatchState)
            if dispatchState == "DONE":
                done = True
            else:
                time.sleep(1)

Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  RUNNING
Search Status:  DONE


In [92]:
r = requests.get(base_url + '/services/search/jobs/' + sid + '/results/',
    headers = { 'Authorization': ('Splunk %s' %session_key)},
    auth=(username, password),
    params={'output_mode': 'json', 'count': 1000},
    verify = False)

In [93]:
r.text

'{"preview":false,"init_offset":0,"messages":[{"type":"INFO","text":"Your timerange was substituted based on your search string"}],"fields":[{"name":"date","type":"str","groupby_rank":"0"},{"name":"hour","type":"str","groupby_rank":"1"},{"name":"brand","type":"str","groupby_rank":"2"},{"name":"device","type":"str","groupby_rank":"3"},{"name":"zero_breaks"},{"name":"some_breaks"}],"results":[{"date":"11/30/20","hour":"11/30/20 12:00:00","brand":"ahc","device":"firetv","zero_breaks":"0","some_breaks":"1"},{"date":"11/30/20","hour":"11/30/20 12:00:00","brand":"apl","device":"android","zero_breaks":"0","some_breaks":"2"},{"date":"11/30/20","hour":"11/30/20 12:00:00","brand":"apl","device":"desktop-web","zero_breaks":"0","some_breaks":"3"},{"date":"11/30/20","hour":"11/30/20 12:00:00","brand":"apl","device":"firetv","zero_breaks":"0","some_breaks":"4"},{"date":"11/30/20","hour":"11/30/20 12:00:00","brand":"apl","device":"ios","zero_breaks":"0","some_breaks":"1"},{"date":"11/30/20","hour":"1